<H1> Fraud Detection Model</h1>

<H3><font style="font-family:calibri">The model is implemented to predict the Fraud Transactions from the provided dataset.
    To create the decision model, RandomForestClassifier is used which gives its output on the decision from several features that are considered to be the deciding features for a fraud transaction, such as Incomplete Amount transaction between end-to-end. 
    <Br>The model gave a good accuracy and F1 score in comparison to the original accuracy provided in the dataset 

In [ ]:
import pandas as pd

# Importing the Dataset

In [ ]:
data=pd.read_csv(r"C:\Codes\Python\Data\Fraud.csv")
data.shape

# Sample data

In [ ]:
data.sample(15)

# Data Description

In [ ]:
print(data.describe().apply(lambda s: s.apply('{0:.5f}'.format)))

<H3>First Condition, correcting the 'isFlaggedFraud' value by the given condition:
    <Br>"An illegal attempt in this dataset is an attempt to transfer more than 200000 in a single transaction."
<Br>Hence missing data according to condition is Corrected</H3>

In [ ]:
data.loc[data["amount"] >= 200000, "isFlaggedFraud"] = 1

# Now, creating different Markers to analyse the different kinds of transactions that happened


<H3><font style="font-family:calibri">This segment answers the 3rd question about selecting the features determining fraud transactions. To predict if a Transaction is fraud, the conditions should be like:<Br>
<p style="margin-left:50px";>1. The Transaction Amount was over the limiting value, i.e. 200000 per transaction.
<Br>2. The amount transferred was not equal to the amount received.
<Br>3. Receiver's Balance altered without any change in the sender's account balance (Defined Fraud-case) and vice-versa.
<Br>4. Sender's account debited without any credit to the Receiver.
</p>
<Br>
Thus, the features to be considered here are:
<p style="margin-left:50px";>1. Original Features: amount, oldbalanceOrg, oldbalanceDest, newbalanceOrig, newbalanceDest, isFlaggedFraud, type
<br>2. Extracted Features: origTransfer, destTransfer, CompleteTransfer, SuccessfullySent, CompleteAmtRecieved. 
</p>
</H3>

<H3>Creating Extracted feature to display the net transaction from both Origin and Destination.</H3>

In [ ]:
data['origTransfer']=abs(data.oldbalanceOrg-data.newbalanceOrig)
data['destTransfer']=abs(data.oldbalanceDest-data.newbalanceDest)

<H3>Creating a Flag to mark the transactions that came out to be Complete from end-to-end.</H3>

In [ ]:
data['CompleteTransfer'] = 1
data.loc[ (data.origTransfer != data.destTransfer)
          | (data.amount!=data.origTransfer)
          | (data.amount!=data.destTransfer),  'CompleteTransfer'] = 0

data.loc[data.nameDest.str.contains("M", case=False)
         & ((data.amount == data.origTransfer) | (data.amount == data.destTransfer)) , 'CompleteTransfer'] = 1

<H3>Creating a Flag to mark the successfully sent transactions from the Origin-End.</H3>

In [ ]:
data['SuccessfullySent'] = 1
data.loc[ (data.amount!=data.origTransfer),  'SuccessfullySent'] = 0

data.loc[data.nameDest.str.contains("M", case=False)
         & (data.amount == data.origTransfer), 'SuccessfullySent'] = 1

<H3>Creating a Flag to mark the successfully recieved transacions from the Reciever-End.</H3>

In [ ]:
data['CompleteAmtRecieved'] = 1
data.loc[ (data.amount!=data.destTransfer),  'CompleteAmtRecieved'] = 0

data.loc[data.nameDest.str.contains("M", case=False)
         & (data.amount == data.origTransfer), 'CompleteAmtRecieved'] = 1

<H3>Generating a display for manual check of the Transfers.</H3>

In [ ]:
data[['amount','origTransfer','destTransfer','nameDest','CompleteTransfer','SuccessfullySent','CompleteAmtRecieved']]

<H3>Let us now display the Transactions in different segments: Complete, Incomplete (From either Sender or Reciever End) and Frauds-Faulty Transactions. </H3>

<H3>Creating a Dataset out of the Truely-Completed Transactions.

In [ ]:
true=data.where(data.CompleteTransfer==1)
true=true.dropna()
true.shape

In [ ]:
true.sample(7)

<H3>Creating a Dataset out of the Successfully Sent Transactions.

In [ ]:
sentSuccess=data.where(data.SuccessfullySent==1)
sentSuccess=sentSuccess.dropna()
sentSuccess.shape

In [ ]:
sentSuccess.sample(7)

<H3>Creating a Dataset out of the Successfully Recieved Transactions.

In [ ]:
recieveSuccess = data.where(data.CompleteAmtRecieved == 1)
recieveSuccess = recieveSuccess.dropna()
recieveSuccess.shape

In [ ]:
recieveSuccess.sample(7)

<H3>Creating a Dataset out of all other Transactions including Outliers, incomplete or faulty Transactions.

In [ ]:
faultyTransfer=data.where((data.CompleteTransfer == 0) & (data.CompleteAmtRecieved == 0) & (data.SuccessfullySent==0))
faultyTransfer=faultyTransfer.dropna()
faultyTransfer.shape

In [ ]:
faultyTransfer.sample(16)

<H3>Since we have to find out the Fraud Transactions out of the dataset, we are not considering the Successfully Completed Transactions.
<bR><BR>Therefore, proceeding with the remaining datasets where Incomplete transactions are made, along the faulty transaction data.

In [ ]:
FraudData=data[~data.isin(true)].dropna()
FraudData.shape

In [ ]:
FraudData.where(FraudData.isFraud==1).dropna().sample(7)

<H3>
Current Accuracy of the model based on Provided Data

In [ ]:
from sklearn.metrics import accuracy_score
print('accuracy = ',round(accuracy_score(FraudData.isFraud, FraudData.isFlaggedFraud)*100),'%')

<H3>
    Creating Predictor Model for Fraud data Based on the Data

In [ ]:
FraudData.where(FraudData.isFlaggedFraud==1).dropna()

<H3>
    Create the OneHotEncoder to convert Type from Categorical to Numerical

In [ ]:
FraudDataOHE = pd.get_dummies (FraudData['type'], drop_first=True)

In [ ]:
FraudData=FraudData.join(FraudDataOHE)
FraudData

<H3>Using Random Forest Classifier as the Fraud Predictor

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

<H3>
    Discarding Unnecessary Columns from Predictor

In [ ]:
X = FraudData[['amount','oldbalanceDest','oldbalanceOrg','newbalanceDest','newbalanceOrig','origTransfer','destTransfer', 'CASH_OUT','DEBIT','PAYMENT','TRANSFER']]
Y = FraudData[['isFraud']]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.3)

In [ ]:
print(pd.DataFrame(xtrain))

In [ ]:
classifier= RandomForestClassifier(criterion='gini', random_state=0)
classifier.fit(xtrain, ytrain)

In [ ]:
Y_Pred = classifier.predict(xtest)

<H2><font style="font-family:calibri">Performance Metrics</h2>
    <h3>Analysing the accuracy, f1 score, precision and recall of the Model, it can be considered that it is an efficient model to predict fraud transactions if the necessary parameters are provided.</h3>

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
print('accuracy = ',round(accuracy_score(ytest, Y_Pred)*100,2),'%')
print('precision = ',round(precision_score(ytest, Y_Pred)*100,2),'%')
print('recall = ',round(recall_score(ytest, Y_Pred)*100,2),'%')
print('f1-score = ',round(f1_score(ytest, Y_Pred)*100,2),'%')
print('roc_auc score = ',round(roc_auc_score(ytest, Y_Pred)*100,2),'%\n\n')

from sklearn.metrics import classification_report
print(classification_report(ytest, Y_Pred))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(ytest, Y_Pred)
print(cm)
plt.figure(figsize=(9,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Predicted Fraud')
plt.ylabel('Original Fraud')

<H3> As mentioned above, the features that finally are giving out the correct result for the fraudulant transactions are:<br> 'amount','oldbalanceDest','oldbalanceOrg','newbalanceDest','newbalanceOrig','origTransfer','destTransfer', 'type'.
    
<br> These factors determine whether the transaction is fraud or not on the basis of the given conditions such as the limits-per-transaction, end-to-end complete transaction of amounts, etc. the type of transaction is also considered as the isFraud feature is considered when the Fraudulated user cashes out all of the victim's amount, or such, whereas the balance logs play a vital role for checking out of complete transactions.

<H3>Now mapping the prediction on the complete dataset

In [ ]:
Fraud_Predict = classifier.predict(X)
Fraud_Predict=pd.DataFrame(Fraud_Predict)

In [ ]:
FraudData=FraudData.drop(columns=FraudData[['CASH_OUT','DEBIT','PAYMENT','TRANSFER','origTransfer','destTransfer','CompleteTransfer','SuccessfullySent', 'CompleteAmtRecieved']])

In [ ]:
FraudData['NewPredict']=Fraud_Predict
FraudData=FraudData.dropna()

<H3>Displaying Fraud Predicted Samples which were originally flagged as Fraud

In [ ]:
FraudData.where((FraudData.NewPredict==1) & ((FraudData.isFraud==1) | (FraudData.isFlaggedFraud==1))).dropna()

<H3>Displaying Non-Fraud Predicted Samples

In [ ]:
FraudData.where(FraudData.NewPredict==0).sample(7)

<H3>Displaying Proportion of Frauds in original Dataset

In [ ]:
true=true.drop(columns=true[['origTransfer','destTransfer','CompleteTransfer','SuccessfullySent', 'CompleteAmtRecieved']])

In [ ]:
FinalData= pd.concat([true, FraudData])

In [ ]:
Frauds=FinalData['NewPredict'].value_counts()[1]
Non_Frauds=FinalData['NewPredict'].value_counts()[0]
print('\n\nTotal Fraud Transactions: ',Frauds,' --> ',round(Frauds/(Frauds+Non_Frauds)*100,2),'% of total Transactions')
print('Total Non-Fraud Transactions: ',Non_Frauds,' --> ',round(Non_Frauds/(Frauds+Non_Frauds)*100,2),'% of total Transactions')

<H2>Conclusion:</h2>
<H3>To conclude, it can be inferred that the company can reduce the amount of fraud transactions firstly by not allowing any transaction over the limit at a time. It may also analyse the change of data for the Merchant accounts as well in order to keep a check if any merchant is performing frauds or not. The total endpoint transaction should be preserved in order to reduce incomplete transaction with proper monitoring.</h3>

In [39]:
FinalData= pd.concat([true, FraudData])

In [40]:
Frauds=FinalData['NewPredict'].value_counts()[1]
Non_Frauds=FinalData['NewPredict'].value_counts()[0]
print('\n\nTotal Fraud Transactions: ',Frauds,' --> ',round(Frauds/(Frauds+Non_Frauds)*100,2),'% of total Transactions')
print('Total Non-Fraud Transactions: ',Non_Frauds,' --> ',round(Non_Frauds/(Frauds+Non_Frauds)*100,2),'% of total Transactions')



Total Fraud Transactions:  768  -->  0.08 % of total Transactions
Total Non-Fraud Transactions:  933484  -->  99.92 % of total Transactions


<H2>Conclusion:</h2>
<H3>To conclude, it can be inferred that the company can reduce the amount of fraud transactions firstly by not allowing any transaction over the limit at a time. It may also analyse the change of data for the Merchant accounts as well in order to keep a check if any merchant is performing frauds or not. The total endpoint transaction should be preserved in order to reduce incomplete transaction with proper monitoring.</h3>